In [19]:
analyse_path = r'C:\Users\ayazici\Documents\Analyses'

In [37]:
import scipy.io
import os
import h5py
import numpy as np
from scipy.signal import find_peaks, savgol_filter
import matplotlib.pyplot as plt
# from scipy.stats import sem
from scipy.stats import wilcoxon
import pandas as pd
import json
import jdata as jd

In [11]:
import tkinter as tk
from tkinter import filedialog

class SelectFoldersDialog:
    def __init__(self, master):
        self.master = master
        self.selected_folders = []

        self.label = tk.Label(master, text="Sélectionnez les dossiers :")
        self.label.pack()

        self.add_button = tk.Button(master, text="Ajouter un dossier", command=self.add_folder)
        self.add_button.pack()

        self.confirm_button = tk.Button(master, text="Confirmer la sélection", command=self.confirm_selection)
        self.confirm_button.pack()

    def add_folder(self):
        folder = filedialog.askdirectory()
        if folder:
            self.selected_folders.append(folder)
            print("Dossier ajouté :", folder)

    def confirm_selection(self):
        if self.selected_folders:
            print("Sélection confirmée :", self.selected_folders)
            self.master.destroy()
        else:
            print("Aucun dossier sélectionné. Veuillez ajouter au moins un dossier.")

            # Si aucun dossier n'est sélectionné, vous pouvez choisir de quitter l'application ici
            # self.master.destroy()

def open_select_folders_dialog():
    root = tk.Tk()
    dialog = SelectFoldersDialog(root)
    root.mainloop()

    return dialog.selected_folders  # Retourner les dossiers sélectionnés

# Exemple d'utilisation : ouvrir une fenêtre de dialogue pour sélectionner les dossiers
selected_folders = np.unique(open_select_folders_dialog())

Dossier ajouté : P:/SharedFiles/Abdussamed/Pulvinar_rec_dark_80degs/animal21_a53d1s1_ephys/21a53d1s1_210604_150840
Dossier ajouté : P:/SharedFiles/Abdussamed/Pulvinar_rec_dark_80degs/animal21_a51d1s1_ephys/21a51d1s1_210602_110516
Dossier ajouté : P:/SharedFiles/Abdussamed/Pulvinar_rec_dark_80degs/animal21_a21d1s1_ephys/21a21d1s1_210408_155731
Dossier ajouté : P:/SharedFiles/Abdussamed/Pulvinar_rec_dark_80degs/animal21_a19d1s1_ephys/21a19d1s1_210403_130057
Dossier ajouté : P:/SharedFiles/Abdussamed/Pulvinar_rec_dark_80degs/animal21_a18d1s1_ephys/21a18d1s1_210401_111241
Dossier ajouté : P:/SharedFiles/Abdussamed/Pulvinar_rec_dark_80degs/animal21_a17d1s1_ephys/21a17d1s1_210330_130118
Dossier ajouté : P:/SharedFiles/Abdussamed/Pulvinar_rec_dark_80degs/animal21_a15d1s1_ephys/21a15d1s1_210326_163524
Sélection confirmée : ['P:/SharedFiles/Abdussamed/Pulvinar_rec_dark_80degs/animal21_a53d1s1_ephys/21a53d1s1_210604_150840', 'P:/SharedFiles/Abdussamed/Pulvinar_rec_dark_80degs/animal21_a51d1s1_ep

In [69]:
from nbconvert.preprocessors import ExecutePreprocessor
from nbconvert import NotebookExporter
import nbformat
from tqdm import tqdm  # Importer tqdm pour la barre de progression

# Nombre total de dossiers à analyser
total_folders = len(selected_folders)

# Utiliser tqdm pour afficher une barre de progression
for i, selected_path in enumerate(tqdm(selected_folders, desc="Analyzing folders", unit="folder")):
    executed_notebook = os.path.join(r'C:\Users\gbouvier\BOUVIER', 'Analyze.ipynb')
    result_notebook = os.path.join(analyse_path, f"Analyze_{i+1}.ipynb")  

    jd.save(selected_path, os.path.join(analyse_path, 'path.json'))

    # Charger le notebook à exécuter
    with open(executed_notebook, 'r', encoding='utf-8') as f:
        notebook = nbformat.read(f, as_version=4)

    # Créer un préprocesseur pour exécuter le notebook
    preprocessor = ExecutePreprocessor(timeout=None)

    # Exécuter le notebook
    preprocessor.preprocess(notebook)

    # Exporter le notebook exécuté
    exporter = NotebookExporter()
    body, resources = exporter.from_notebook_node(notebook)

    # Écrire le notebook exécuté dans un fichier
    with open(result_notebook, 'w', encoding='utf-8') as f:
        f.write(body)



Analyzing folders: 100%|██████████| 7/7 [48:23<00:00, 414.79s/folder]


In [70]:
AllData = {}

data_folder = os.path.join(analyse_path, 'Data')

for i, file in enumerate(tqdm(sorted(os.listdir(data_folder)), desc="Data loading", unit="folder")):
    AllData[file.split('_')[1]] = jd.load(os.path.join(data_folder, file))

Data loading: 100%|██████████| 7/7 [17:34<00:00, 150.60s/folder]


In [71]:
save = True

s=15

for pos_title, posOrientation in zip(['Mediolateral', 'Anteroposterior'], ['ML_pos', 'AP_pos']):
    for condition in ['first', 'second']:
        for direction in ['CW','CCW']:
            for animal in AllData:
                pos = AllData[animal]['informative_data'][posOrientation]
                Nclust = AllData[animal]['SUA_data']['Nclust']
                AllDepth = AllData[animal]['MUA_data']['AllDepth']
                vMI = AllData[animal]['Statistics_data']['vMI']

                plt.scatter(pos*np.ones(Nclust), AllDepth, c=vMI[condition][direction], cmap='coolwarm', s=s)

            plt.gca().invert_yaxis()

            plt.colorbar(label=f"{direction} modulation index")

            plt.xlabel(f"{pos_title} position (mm)")
            plt.ylabel("Depth (µm)")
            plt.title(f"{direction} modulation of units, {condition} condition")

            if save:
                direction_modulation_folder = os.path.join(analyse_path, 'Direction_modulation')
                os.makedirs(direction_modulation_folder, exist_ok=True)
                plt.savefig(os.path.join(direction_modulation_folder , f"{direction}_modulation_{condition}_{pos_title}.png"))
                plt.close()
            else:
                plt.show()

In [72]:
save = True

s=15

for pos_title, posOrientation in zip(['Mediolateral', 'Anteroposterior'], ['ML_pos', 'AP_pos']):
    for condition in ['first', 'second']:
        for animal in AllData:
            pos = AllData[animal]['informative_data'][posOrientation]
            Nclust = AllData[animal]['SUA_data']['Nclust']
            AllDepth = AllData[animal]['MUA_data']['AllDepth']
            dirMI = AllData[animal]['Statistics_data']['dirMI']

            plt.scatter(pos*np.ones(Nclust), AllDepth, c=dirMI[condition], cmap='coolwarm', s=s)

        plt.gca().invert_yaxis()

        plt.colorbar(label=f"Direction Modulation Index\n(CW - CCW) / (CW + CCW)")

        plt.xlabel("Mediolateral position (mm)")
        plt.ylabel("Depth (µm)")
        plt.title(f"CW vs CCW preference")

        if save:
            direction_preference_folder = os.path.join(analyse_path, 'Direction_preference')
            os.makedirs(direction_preference_folder, exist_ok=True)
            plt.savefig(os.path.join(direction_preference_folder , f"Direction_preference_{condition}_{pos_title}.png"))
            plt.close()
        else:
            plt.show()